In [1]:
import pickle 
import numpy as np 
import pandas as pd 
from sklearn.metrics import r2_score 
import matplotlib.pyplot as plt 

In [2]:
#load saved data 
obj = pd.read_pickle('data/update2_pickle.pickle')

# Profit Calculations

## Helper Functions 

In [3]:
#find index 
name_array = np.array(obj[0])
all_predicted_values = obj[1]

#find relevant index from saved data
def index_finder(csv_name):

    #find the index from the loaded object
    idx = np.where(name_array == csv_name)[0][0]
    
    return idx 

#get the predictions from Saved Data
def preds(csv_name):
    
    #pull index predicted values
    values = pd.DataFrame(all_predicted_values[index_finder(csv_name)], 
                                   columns = ['Open', 'High', 'Low', 'Close'])
    
    #opening values
    predicted_values = np.array(values.iloc[:,0])
    
    return predicted_values


#Get the actual values from the saved data 
def actual(csv_name):
    
    #get actual opening values 
    df = pd.read_csv('data/{}'.format(csv_name))
    actual_vals = np.array(df.iloc[:,1])
    
    return actual_vals

## Calculate Buy Decisions and Averages

In [4]:
#running average, std deviations for previous k hours (opening price) 
def moving_averages(csv_name, k, cut_off):
    
    """
    For specified csv:
        - return the weighted averages
        - std deviations by window size k
        
    In order to avoid unncessary risk, you can specifya period (say hour 85) to cut off buying
    
        
    Output: Dataframe with Actual Values within the last 100 Hours of Trading
            Predicted Values
            Moving Averages for specified-k Day Lag 
            Std Deviations for the Moving Averages 
    """
    
    #load actual/predicted values
    actual_values = actual(csv_name)[-100 - k - 2:]
    predicted_values = preds(csv_name)
     
    #start calculations for the moving averages 
    moving_avs = []
    moving_stds = []
    
    for i,j in enumerate(predicted_values):
    
        #values needed
        block = actual_values[i:k+i]

        #compute mean, std append
        avg, std = np.average(block), np.std(block)
        moving_avs.append(avg)
        moving_stds.append(std)
    
    
    #recast to be the final 100 hours 
    actual_values = actual_values[k+2:]
    #print(len(actual_values))
    
    #For Buy Decision: Predicted - Moving Averages <0 and less than std of mean 
    buy_decision = []
    net_values = []
    
    #Calculate
    for i in range(100):
        
        #Differences between predictions, moving average of actual values 
        difference = predicted_values[i] - moving_avs[i]
        
        #if you buy based on the prediction, this is the difference between predicted and actual
        actual_worth = predicted_values[i] - actual_values[i]
        
        
        #ther predicted value is less than the standard deviation
        if (difference < 0) & (abs(difference) > moving_stds[i]) & (i <= cut_off):  
            buy_decision.append(1)
            net_values.append(actual_worth)
            
        else: 
            buy_decision.append(0)
            net_values.append(0)
    
    
    return pd.DataFrame({'Actual Values':actual_values, 
                         'Predicted Values':predicted_values,
                        'Moving Averages for {} Day Lag'.format(k):moving_avs, 
                        'Std Deviations':moving_stds, 
                        'Buy Coin?': buy_decision, 
                        'Difference': net_values})

## Calculate Sell Decisions

In [5]:
def buy_sell(df):
    
    """
    Given weighted averages, standard deviations, choose stocks purchased at indices within
    the last 100 hours to sell off. 
    
    Sell decision is where difference between predicted value of coin is at least one standard 
    deviation above the price the coin was purchased at 
    
    Returns a dataframe of the trades made and their respective statistics, assets held after 
    """
    
    sell_time = []
    sold_array = []
    baseline = []
    
    buy_decisions = np.where(df.iloc[:,4] == 1)[0]
    
    
    #in subsequent time periods, we take sell greedily 
    #1st time the predicted value is above 1 std, we sell 
    
    for time in buy_decisions: 
        
        for i in range(time, df.shape[0]): 
            
            #predicted value one standard deviation above actual value 
            #predicted pattern at hour i 
            predicted_value = df.iloc[i, 1]

            #purchase price 
            buy_time_actual = df.iloc[time, 0]

            #running adjusted std dev
            std_dev_actual = df.iloc[i, 3]

            #measure the potential difference between prediced price and the purchase, 
            potential_difference = predicted_value - buy_time_actual

            # if the potential difference is greater than 1 std at current time, sell at open else hold
            if potential_difference > std_dev_actual: 

                #sell point
                sell_point = df.iloc[i,0]
                cash_flow = sell_point-buy_time_actual
                baseline.append(cash_flow)

                sold_array.append(time)
                sell_time.append(i)
                break
    
    #DF Trades  
    rev_df = pd.DataFrame({'Asset Purchased at time t':sold_array, 
                       'Sold at Time':sell_time, 'Net Revenue': baseline})
    
    profitable_trades_ratio = len(np.where(np.array(baseline) > 0)[0])/len(baseline)
    
    
    #Total Earnings From Trades
    total_earnings = np.sum(baseline)
    
    #Purchased Assets Still Held 
    assets_held = list(set(buy_decisions) - set(sold_array))
    
    #Total Value of Assets Held
    assets_held_value = np.sum([df.iloc[i,0] for i in assets_held])
    
    
    return [rev_df, profitable_trades_ratio, total_earnings, assets_held, assets_held_value]

In [6]:
#profit = 0
#held = 0

#Money made
for i in obj[0]:
    data = moving_averages(i,25,85)
    frame, performance = buy_sell(data)[0], buy_sell(data)[2]
    print(frame, performance)
    #profit += performance
    
#Value of Assets Still Held 
# for i in obj[0]:
#     data = moving_averages(i,25,85)
#     asset = buy_sell(data)[4]
#     held += asset
    
# print(profit)
# print(held)

   Asset Purchased at time t  Sold at Time  Net Revenue
0                          1             2          0.2
1                          9            11          0.6
2                         27            32          0.5
3                         28            32          0.7
4                         29            32          0.2
5                         30            32          0.1
6                         31            32          0.2
7                         33            36          1.4
8                         41            42         -0.8
9                         47            48         -0.3 2.799999999999997
   Asset Purchased at time t  Sold at Time  Net Revenue
0                          1             2        -0.73
1                          6            12        -0.28
2                         10            12        -0.57
3                         15            16         0.03
4                         28            29         1.18
5                         30  

   Asset Purchased at time t  Sold at Time  Net Revenue
0                          5            25      -0.0010
1                          7            25      -0.0011
2                         32            42      -0.0007
3                         33            39       0.0006
4                         34            42      -0.0010
5                         35            42      -0.0008
6                         36            42      -0.0018
7                         80            96       0.0022 -0.003599999999999992
   Asset Purchased at time t  Sold at Time  Net Revenue
0                          1            18         0.32
1                         17            26         0.14
2                         39            42         0.34
3                         40            42         0.26
4                         41            44        -0.13 0.9299999999999997
    Asset Purchased at time t  Sold at Time  Net Revenue
0                          19            21      -0.0116
1    

    Asset Purchased at time t  Sold at Time  Net Revenue
0                           0             7      -0.0055
1                           1             7      -0.0054
2                           4             7      -0.0023
3                           5             7      -0.0114
4                           6             7      -0.0029
5                           9            10       0.0052
6                          15            19       0.0015
7                          18            22       0.0006
8                          30            31      -0.0052
9                          33            34       0.0034
10                         35            39       0.0078
11                         37            41       0.0108 -0.0033999999999998476
    Asset Purchased at time t  Sold at Time  Net Revenue
0                           0            20     -0.00001
1                           1            20      0.00002
2                           4             5      0.00021
3       

   Asset Purchased at time t  Sold at Time  Net Revenue
0                          3             6       0.0011
1                          4            10      -0.0001
2                          5            10      -0.0004
3                          8            10       0.0002
4                         16            19       0.0003
5                         33            35       0.0016
6                         41            43      -0.0005 0.0022000000000000075
    Asset Purchased at time t  Sold at Time  Net Revenue
0                           1             2       0.0010
1                           4             7      -0.0052
2                           9            13       0.0026
3                          10            18       0.0041
4                          16            18       0.0054
5                          17            18       0.0015
6                          32            36       0.0019
7                          33            36       0.0035
8                

    Asset Purchased at time t  Sold at Time  Net Revenue
0                           3            12          3.5
1                           4            12          2.2
2                           5            12          1.8
3                           9            12          4.4
4                          10            12          3.1
5                          11            12          2.5
6                          27            37          4.2
7                          30            38          2.5
8                          31            37          4.0
9                          32            38          1.8
10                         33            37          4.6
11                         34            38          1.4
12                         35            38          0.9 36.89999999999992
    Asset Purchased at time t  Sold at Time  Net Revenue
0                           4             5       0.0016
1                           8            13       0.0048
2            

...

   Asset Purchased at time t  Sold at Time  Net Revenue
0                          0             3        0.045
1                          1            20        0.005
2                          8            20        0.014
3                         35            36        0.010
4                         53            54        0.073
5                         84            89       -0.019 0.12800000000000056
    Asset Purchased at time t  Sold at Time  Net Revenue
0                           0            17       -0.003
1                           1             2       -0.003
2                           3             6        0.004
3                           4             6        0.002
4                           7            15        0.005
5                          10            13        0.006
6                          11            15        0.005
7                          28            32        0.002
8                          63            72        0.000
9                 

   Asset Purchased at time t  Sold at Time  Net Revenue
0                          0             4    -0.000425
1                          1             5     0.008484
2                          2             4    -0.000645
3                          3             5     0.006012
4                          6             7    -0.005293
5                          8            12    -0.002352
6                         14            19     0.004295
7                         15            19     0.004423 0.01449900000000004
   Asset Purchased at time t  Sold at Time  Net Revenue
0                          0             3         0.00
1                          1             3        -0.02
2                          4             7         0.00
3                          5            10         0.04
4                          6             7        -0.01
5                          9            10         0.02
6                         49            56         0.08
7                         52

    Asset Purchased at time t  Sold at Time  Net Revenue
0                           0             7      -0.0048
1                           1            19      -0.0020
2                           2             7      -0.0027
3                           3            19      -0.0038
4                           8            19      -0.0019
5                           9            19      -0.0011
6                          15            18       0.0135
7                          29            41       0.0135
8                          32            41       0.0134
9                          33            36      -0.0021
10                         34            41       0.0141
11                         38            41       0.0151
12                         85            90      -0.0227 0.028500000000000025
    Asset Purchased at time t  Sold at Time  Net Revenue
0                           4             7      -0.0014
1                           5            15       0.0034
2         

   Asset Purchased at time t  Sold at Time  Net Revenue
0                          0            26     -0.00144
1                         12            15      0.00003
2                         19            26     -0.00098
3                         24            26     -0.00165
4                         27            34      0.00017
5                         28            31      0.00033
6                         29            37      0.00060
7                         35            37      0.00019
8                         41            46      0.00055 -0.0021999999999999797
   Asset Purchased at time t  Sold at Time  Net Revenue
0                          1             8        0.002
1                          4             8       -0.003
2                          5             8       -0.002
3                         12            13        0.003
4                         14            17        0.001
5                         23            24       -0.001
6                        

    Asset Purchased at time t  Sold at Time  Net Revenue
0                           5             7       0.0001
1                           6             7       0.0010
2                          12            14      -0.0001
3                          13            14      -0.0022
4                          30            34       0.0064
5                          32            34       0.0066
6                          42            51       0.0074
7                          79            85      -0.0022
8                          80            85      -0.0033
9                          81            85      -0.0007
10                         83            85      -0.0062 0.006800000000000028
   Asset Purchased at time t  Sold at Time  Net Revenue
0                          0             3         0.62
1                          1             3         0.00
2                         16            19         0.87
3                         85            90        -1.12 0.3700000000000

In [13]:
profit_usd = 0

for i in usd_only.iloc[:,0]:
    i = '{}.csv'.format(i)
    data = moving_averages(i,25,85)
    frame, performance = buy_sell(data)[0], buy_sell(data)[2]
    
    #print(i, frame, performance)
    profit_usd += performance
    
held_usd = 0

for i in usd_only.iloc[:,0]:
    i = '{}.csv'.format(i)
    data = moving_averages(i,25,85)
    performance = buy_sell(data)[4]
    held_usd += performance
    
print(profit_usd)
print(held_usd)

4308.213076000004
2.8678


# Modeling 

In [9]:
#first index is names, second index is predicted, third is r squared, fourth is mape
#create array of coins I care about 

csv_array = ['BTCUSD.csv', 'ETHUSD.csv', 'SOLUSD.csv', 'ADAUSD.csv', 'XRPBUSD.csv',
             'DOTUSD.csv', 'AVAXUSD.csv', 'DOGEUSD.csv', 'SHIBUSD.csv', 'MATICUSD.csv']

index_array = []
name_array = np.array(obj[0])
r2_array = []
opening_mape = []

print(name_array)

for i in name_array: 
    #load indices for the coin pairs that we need 
    idx = np.where(name_array == i)[0][0]
    index_array.append(idx)
    
    #get data frame that we need for coin pairs 
    df = pd.read_csv('data/{}'.format(i))

    
    #predicted values 
    values = pd.DataFrame(obj[1][idx], columns = ['Predicted Open', 'High', 'Low', 'Close'])
    predicted_values = values.iloc[:,0]
    
    #actual values
    actual_values = df.iloc[-100:]
    print(actual_values)
    actual_values = actual_values.iloc[:,1]
    
    r2 = r2_score(actual_values,predicted_values)
    r2_array.append(r2)
    
    mape_score = obj[3][idx][1]
    opening_mape.append(mape_score)
    
    #r2 scores
    #print("name:{}, \n MAPE Open:{}, \n r2 score:{}".format(obj[0][idx], obj[3][idx][1], r2))
    
    

['LTCBUSD.csv' 'EGLDBUSD.csv' 'GALAUSD.csv' 'MIRBUSD.csv' 'CELRBUSD.csv'
 'PYRBUSD.csv' 'POWRUSD.csv' 'XVSBUSD.csv' 'BNBBUSD.csv' 'YFIBUSD.csv'
 'ACMBUSD.csv' 'EOSUSD.csv' 'BAKEBUSD.csv' 'QTUMBUSD.csv' 'FISBUSD.csv'
 'ARBUSD.csv' 'MLNUSD.csv' 'COVALUSD.csv' 'VETBUSD.csv' 'ERNBUSD.csv'
 'IOTXBUSD.csv' 'ELFBUSD.csv' 'DOTUSDT.csv' 'LAZIOBUSD.csv' 'MDTUSD.csv'
 'LINABUSD.csv' 'SNXBUSD.csv' 'ANKRBUSD.csv' 'HIVEBUSD.csv' 'RENBUSD.csv'
 'BICOBUSD.csv' 'ATOMBUSD.csv' 'DARBUSD.csv' 'POLYBUSD.csv' 'MATICUSD.csv'
 'DOGEUSD.csv' 'RARIUSD.csv' 'MANAUSD.csv' 'COMPBUSD.csv' 'INJBUSD.csv'
 'THETABUSD.csv' 'ZECUSD.csv' 'DIABUSD.csv' 'ALGOUSD.csv' 'AMPUSD.csv'
 'CTSIBUSD.csv' 'FTTBUSD.csv' 'CELOBUSD.csv' 'BADGERBUSD.csv' 'RENUSD.csv'
 'BALUSD.csv' 'NKNUSD.csv' 'OOKIBUSD.csv' 'BADGERUSD.csv' 'DYDXBUSD.csv'
 'SOLBUSD.csv' 'LPTUSD.csv' 'CROUSD.csv' 'ZRXBUSD.csv' 'JSTBUSD.csv'
 'HNTBUSD.csv' 'BCHUSD.csv' 'ETHUSD.csv' 'KRLUSD.csv' 'BATUSD.csv'
 'AERGOBUSD.csv' 'PLAUSD.csv' 'AVAXUSDT.csv' 'HIGHBUSD.csv' 'BCHB

                 datetime  LINABUSD_open  LINABUSD_high  LINABUSD_low  \
3524  2022-05-27 20:00:00        0.00868        0.00871       0.00845   
3525  2022-05-27 21:00:00        0.00847        0.00853       0.00829   
3526  2022-05-27 22:00:00        0.00839        0.00859       0.00832   
3527  2022-05-27 23:00:00        0.00858        0.00858       0.00846   
3528  2022-05-28 00:00:00        0.00854        0.00872       0.00852   
...                   ...            ...            ...           ...   
3619  2022-05-31 19:00:00        0.01102        0.01154       0.01095   
3620  2022-05-31 20:00:00        0.01120        0.01143       0.01111   
3621  2022-05-31 21:00:00        0.01115        0.01122       0.01084   
3622  2022-05-31 22:00:00        0.01101        0.01101       0.01047   
3623  2022-05-31 23:00:00        0.01054        0.01059       0.01034   

      LINABUSD_close  
3524         0.00846  
3525         0.00842  
3526         0.00859  
3527         0.00856  
3528    

                 datetime  SOLBUSD_open  SOLBUSD_high  SOLBUSD_low  \
3524  2022-05-27 20:00:00         41.98         42.44        40.93   
3525  2022-05-27 21:00:00         41.04         41.38        40.14   
3526  2022-05-27 22:00:00         40.72         41.35        40.34   
3527  2022-05-27 23:00:00         41.29         41.62        41.03   
3528  2022-05-28 00:00:00         41.42         42.12        41.28   
...                   ...           ...           ...          ...   
3619  2022-05-31 19:00:00         45.74         46.09        45.06   
3620  2022-05-31 20:00:00         45.96         46.00        45.47   
3621  2022-05-31 21:00:00         45.89         45.92        45.15   
3622  2022-05-31 22:00:00         45.55         45.58        44.65   
3623  2022-05-31 23:00:00         44.73         44.97        44.31   

      SOLBUSD_close  
3524          41.01  
3525          40.72  
3526          41.27  
3527          41.42  
3528          42.06  
...             ...  
3619 

                 datetime  SCRTBUSD_open  SCRTBUSD_high  SCRTBUSD_low  \
3524  2022-05-27 20:00:00          1.653          1.660         1.630   
3525  2022-05-27 21:00:00          1.630          1.633         1.615   
3526  2022-05-27 22:00:00          1.617          1.646         1.603   
3527  2022-05-27 23:00:00          1.640          1.642         1.634   
3528  2022-05-28 00:00:00          1.638          1.650         1.630   
...                   ...            ...            ...           ...   
3619  2022-05-31 19:00:00          1.661          1.687         1.656   
3620  2022-05-31 20:00:00          1.688          1.688         1.678   
3621  2022-05-31 21:00:00          1.682          1.682         1.674   
3622  2022-05-31 22:00:00          1.677          1.679         1.658   
3623  2022-05-31 23:00:00          1.660          1.703         1.650   

      SCRTBUSD_close  
3524           1.630  
3525           1.620  
3526           1.638  
3527           1.640  
3528    

                 datetime  FORTHBUSD_open  FORTHBUSD_high  FORTHBUSD_low  \
3524  2022-05-27 20:00:00            2.92            2.93           2.89   
3525  2022-05-27 21:00:00            2.88            2.88           2.81   
3526  2022-05-27 22:00:00            2.84            2.88           2.81   
3527  2022-05-27 23:00:00            2.90            2.90           2.88   
3528  2022-05-28 00:00:00            2.89            2.92           2.89   
...                   ...             ...             ...            ...   
3619  2022-05-31 19:00:00            3.48            3.50           3.43   
3620  2022-05-31 20:00:00            3.50            3.50           3.48   
3621  2022-05-31 21:00:00            3.48            3.48           3.43   
3622  2022-05-31 22:00:00            3.42            3.42           3.39   
3623  2022-05-31 23:00:00            3.42            3.43           3.39   

      FORTHBUSD_close  
3524             2.89  
3525             2.84  
3526           

...

                 datetime  CTSIUSD_open  CTSIUSD_high  CTSIUSD_low  \
3524  2022-05-27 20:00:00        0.1776        0.1797       0.1772   
3525  2022-05-27 21:00:00        0.1780        0.1790       0.1761   
3526  2022-05-27 22:00:00        0.1761        0.1785       0.1759   
3527  2022-05-27 23:00:00        0.1774        0.1774       0.1737   
3528  2022-05-28 00:00:00        0.1756        0.1813       0.1746   
...                   ...           ...           ...          ...   
3619  2022-05-31 19:00:00        0.2083        0.2139       0.2076   
3620  2022-05-31 20:00:00        0.2109        0.2116       0.2085   
3621  2022-05-31 21:00:00        0.2112        0.2117       0.2089   
3622  2022-05-31 22:00:00        0.2099        0.2162       0.2097   
3623  2022-05-31 23:00:00        0.2145        0.2145       0.2094   

      CTSIUSD_close  
3524         0.1776  
3525         0.1762  
3526         0.1770  
3527         0.1759  
3528         0.1812  
...             ...  
3619 

                 datetime  AXSUSD_open  AXSUSD_high  AXSUSD_low  AXSUSD_close
3524  2022-05-27 20:00:00        18.49        18.70       18.39         18.49
3525  2022-05-27 21:00:00        18.49        18.65       18.35         18.36
3526  2022-05-27 22:00:00        18.36        18.54       18.29         18.33
3527  2022-05-27 23:00:00        18.35        18.36       18.07         18.27
3528  2022-05-28 00:00:00        18.27        18.59       18.15         18.56
...                   ...          ...          ...         ...           ...
3619  2022-05-31 19:00:00        22.91        23.15       22.60         22.78
3620  2022-05-31 20:00:00        22.85        23.12       22.70         23.12
3621  2022-05-31 21:00:00        23.11        23.18       22.82         22.82
3622  2022-05-31 22:00:00        22.83        23.16       22.68         23.16
3623  2022-05-31 23:00:00        23.15        23.95       23.05         23.86

[100 rows x 5 columns]
                 datetime  SANDBUSD_open

                 datetime  TRXTUSD_open  TRXTUSD_high  TRXTUSD_low  \
3524  2022-05-27 20:00:00       0.08051       0.08097      0.07950   
3525  2022-05-27 21:00:00       0.07961       0.08016      0.07892   
3526  2022-05-27 22:00:00       0.07922       0.08021      0.07891   
3527  2022-05-27 23:00:00       0.08006       0.08039      0.07973   
3528  2022-05-28 00:00:00       0.08023       0.08036      0.07971   
...                   ...           ...           ...          ...   
3619  2022-05-31 19:00:00       0.08432       0.08432      0.08257   
3620  2022-05-31 20:00:00       0.08363       0.08363      0.08240   
3621  2022-05-31 21:00:00       0.08272       0.08278      0.08202   
3622  2022-05-31 22:00:00       0.08244       0.08277      0.08204   
3623  2022-05-31 23:00:00       0.08284       0.08385      0.08283   

      TRXTUSD_close  
3524        0.07960  
3525        0.07935  
3526        0.08005  
3527        0.08020  
3528        0.07993  
...             ...  
3619 

                 datetime  BTCTUSD_open  BTCTUSD_high  BTCTUSD_low  \
3524  2022-05-27 20:00:00      28740.33      28740.33     28481.48   
3525  2022-05-27 21:00:00      28672.64      28710.74     28594.25   
3526  2022-05-27 22:00:00      28612.66      28810.90     28584.21   
3527  2022-05-27 23:00:00      28822.56      28847.79     28774.27   
3528  2022-05-28 00:00:00      28828.24      28885.37     28828.24   
...                   ...           ...           ...          ...   
3619  2022-05-31 19:00:00      31702.92      31935.47     31691.56   
3620  2022-05-31 20:00:00      31851.19      31885.72     31844.10   
3621  2022-05-31 21:00:00      31880.18      31880.18     31711.84   
3622  2022-05-31 22:00:00      31773.25      31773.25     31610.05   
3623  2022-05-31 23:00:00      31613.81      31619.87     31505.88   

      BTCTUSD_close  
3524       28575.85  
3525       28649.38  
3526       28767.37  
3527       28847.79  
3528       28850.70  
...             ...  
3619 

                 datetime  AAVEUSD_open  AAVEUSD_high  AAVEUSD_low  \
3524  2022-05-27 20:00:00         92.62         94.42        92.23   
3525  2022-05-27 21:00:00         93.58         93.84        92.32   
3526  2022-05-27 22:00:00         92.49         94.16        91.97   
3527  2022-05-27 23:00:00         93.47         93.57        91.38   
3528  2022-05-28 00:00:00         92.30         94.81        91.80   
...                   ...           ...           ...          ...   
3619  2022-05-31 19:00:00        113.53        113.93       112.01   
3620  2022-05-31 20:00:00        112.51        114.08       112.51   
3621  2022-05-31 21:00:00        114.15        114.15       112.49   
3622  2022-05-31 22:00:00        112.52        114.22       112.45   
3623  2022-05-31 23:00:00        114.22        114.82       112.56   

      AAVEUSD_close  
3524          93.31  
3525          92.43  
3526          93.46  
3527          92.29  
3528          94.68  
...             ...  
3619 

In [10]:
##Coins with Highest R2 Scores

coins = [i.split('.')[0] for i in name_array]
# coins


results_df = pd.DataFrame({'Coin':coins, 'R2 Score':r2_array, 'MAPE':opening_mape}, 
                          columns = ['Coin', 'R2 Score', 'MAPE'])

results_df
top_25 = results_df.sort_values(by = ['R2 Score'], ascending = False)[0:25]
top_25

,Coin,R2 Score,MAPE
309,ADAUSD,0.944335,0.019575
167,ADABUSD,0.943310,0.020126
179,XLMUSD,0.941716,0.010710
200,XLMBUSD,0.938424,0.011709
325,ETHTUSD,0.933255,0.008584
288,BTCTUSD,0.927344,0.007018
120,AAVEBUSD,0.923814,0.019372
231,WBTCUSD,0.922221,0.008812
254,AVABUSD,0.918940,0.013238
1,EGLDBUSD,0.918745,0.020303


In [11]:
##Coins with Highest R2 Scores - only in USD

# condition1 = results_df['Coin'].str.contains('BUSD') == False
# condition2 = results_df['Coin'].str.contains('USDC') == False
# condition3 = results_df['Coin'].str.contains('TUSD') == False
# condition4 = results_df['Coin'].str.contains('USDT') == False


usd_only = results_df[~results_df.Coin.str.contains('|'.join(['BUSD']))]
usd_only = usd_only[~usd_only.Coin.str.contains('|'.join(['USDC']))]
usd_only = usd_only[~usd_only.Coin.str.contains('|'.join(['TUSD']))]
usd_only = usd_only[~usd_only.Coin.str.contains('|'.join(['USDT']))]


#usd_only = results_df[condition1 | condition2 | condition3| condition4]

top_usd = usd_only.sort_values(by = ['R2 Score'], ascending = False)[0:25]
top_usd

,Coin,R2 Score,MAPE
309,ADAUSD,0.944335,0.019575
179,XLMUSD,0.941716,0.010710
231,WBTCUSD,0.922221,0.008812
306,UNIUSD,0.907897,0.018002
146,DASHUSD,0.906155,0.012360
62,ETHUSD,0.904955,0.010218
61,BCHUSD,0.899554,0.012079
322,AAVEUSD,0.897348,0.020937
180,BTCUSD,0.895879,0.008366
72,ENSUSD,0.890785,0.021840


In [ ]:
# actual_values = pd.read_csv('data/ADAUSD.csv')
# labels = actual_values.iloc[-100:,0]
# final = [i.split('2022')[1][1::] for i in labels]

# Visualize the Top 25 ($R^2$) Coin/USD Pairs 

In [ ]:
#PLOTTING out opening values series 
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns


name_array = np.array([i.split('.')[0] for i in name_array])
#print(name_array)

for i,j in enumerate(top_usd['Coin']):
    
    #print(j)
    
    #get opening actual values 
    actual_values = pd.read_csv('data/{}.csv'.format(j))
    a_valz = actual_values.iloc[-100:,1]
    
    #get xtick labels
    labels = actual_values.iloc[-100:,0]
    final = [i.split('2022')[1][1::] for i in labels]
    
    #get index from pickle file for predicted opening values  
    idx = np.where(name_array == j)[0][0]
    predicted_values = pd.DataFrame(obj[1][idx], columns = ['Close', 'Predicted Open', 'High', 'Low'])
    predicted_values = predicted_values.iloc[:,1]
    
    #create plot for the opening values 
    #remove top and rightmost spines to clear things up 
    mpl.rcParams["axes.spines.right"] = False
    mpl.rcParams["axes.spines.top"] = False
    
    #LINES
    plt.plot(range(100), a_valz, color = 'black', label = 'Actual Opening Value')
    plt.plot(range(100), predicted_values, linestyle = (0, (1, 1)), color = 'deeppink', alpha = 1, label = 'Predicted Opening Value')
    
    #AXES
    plt.title('Actual vs Predicted Opening Values for ${}/USD in Final 100 Hours of Trading'.format(j.split("USD")[0]))
    plt.rcParams['xtick.labelsize']= 10
    plt.xticks(np.arange(0,100,5), labels = final[::5], rotation = 90)
    plt.xlabel('Hour Steps')
    plt.ylabel('Price ($USD)')
    plt.grid(color = 'dimgray', alpha = 0.6)
    plt.legend(bbox_to_anchor=(1, 0.9))
    plt.plot()
    
    plt.savefig('figures/no_trades/{} Trading.png'.format(j), bbox_inches = 'tight')
    
    plt.close()
    
    

# Visualize the Top 25 ($R^2$) Coin Pairs 

In [ ]:
#PLOTTING out opening values series 
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns


name_array = np.array([i.split('.')[0] for i in name_array])
#print(name_array)

for i,j in enumerate(top_25['Coin']):
    
    #print(j)
    
    #get opening actual values 
    actual_values = pd.read_csv('data/{}.csv'.format(j))
    a_valz = actual_values.iloc[-100:,1]
    
    #get xtick labels
    labels = actual_values.iloc[-100:,0]
    final = [i.split('2022')[1][1::] for i in labels]
    
    #get index from pickle file for predicted opening values  
    idx = np.where(name_array == j)[0][0]
    predicted_values = pd.DataFrame(obj[1][idx], columns = ['Close', 'Predicted Open', 'High', 'Low'])
    predicted_values = predicted_values.iloc[:,1]
    
    #create plot for the opening values 
    #remove top and rightmost spines to clear things up 
    mpl.rcParams["axes.spines.right"] = False
    mpl.rcParams["axes.spines.top"] = False
    
    #LINES
    plt.plot(range(100), a_valz, color = 'black', label = 'Actual Opening Value')
    plt.plot(range(100), predicted_values, linestyle = (0, (1, 1)), color = 'deeppink', alpha = 1, label = 'Predicted Opening Value')
    
    #AXES
    plt.title('Actual vs Predicted Opening Values for ${} in Final 100 Hours of Trading'.format(j))
    plt.rcParams['xtick.labelsize']= 10
    plt.xticks(np.arange(0,100,5), labels = final[::5], rotation = 90)
    plt.xlabel('Hour Steps')
    plt.ylabel('Price')
    plt.grid(color = 'dimgray', alpha = 0.6)
    plt.legend(bbox_to_anchor=(1, 0.9))
    plt.plot()
    
    plt.savefig('figures/no_trades/{} Trading.png'.format(j), bbox_inches = 'tight')
    
    plt.close()
    

# Bottom 25 ($R^2$) Coin/USD Pairs

In [ ]:
bottom_usd = usd_only.sort_values(by = ['R2 Score'], ascending = True)[0:25]

for i,j in enumerate(bottom_usd['Coin']):
    
    #print(j)
    
    #get opening actual values 
    actual_values = pd.read_csv('data/{}.csv'.format(j))
    a_valz = actual_values.iloc[-100:,1]
    
    #get xtick labels
    labels = actual_values.iloc[-100:,0]
    final = [i.split('2022')[1][1::] for i in labels]
    
    #get index from pickle file for predicted opening values  
    idx = np.where(name_array == j)[0][0]
    predicted_values = pd.DataFrame(obj[1][idx], columns = ['Close', 'Predicted Open', 'High', 'Low'])
    predicted_values = predicted_values.iloc[:,1]
    
    #create plot for the opening values 
    #remove top and rightmost spines to clear things up 
    mpl.rcParams["axes.spines.right"] = False
    mpl.rcParams["axes.spines.top"] = False
    
    #LINES
    plt.plot(range(100), a_valz, color = 'black', label = 'Actual Opening Value')
    plt.plot(range(100), predicted_values, linestyle = (0, (1, 1)), color = 'deeppink', alpha = 1, label = 'Predicted Opening Value')
    
    #AXES
    plt.title('Actual vs Predicted Opening Values for ${} in Final 100 Hours of Trading'.format(j))
    plt.rcParams['xtick.labelsize']= 10
    plt.xticks(np.arange(0,100,5), labels = final[::5], rotation = 90)
    plt.xlabel('Hour Steps')
    plt.ylabel('Price')
    plt.grid(color = 'dimgray', alpha = 0.6)
    plt.legend(bbox_to_anchor=(1, 0.9))
    plt.show()

## Buy/Sell Decision Visuals

In [ ]:
name_array = np.array(obj[0])


for i, j in enumerate(top_usd['Coin']):
    
    dat = moving_averages('{}.csv'.format(j), 25, 85)
    decisions_df = buy_sell(dat)[0]
    sell_decisions = decisions_df.iloc[:,1]
 
    
    buy_decisions = np.where(dat.iloc[:,4] == 1)[0]
    
    #get opening actual values 
    actual_values = pd.read_csv('data/{}.csv'.format(j))
    a_valz = actual_values.iloc[-100:,1]
    
    #get xtick labels
    labels = actual_values.iloc[-100:,0]
    final = [i.split('2022')[1][1::] for i in labels]
    
    #get index from pickle file for predicted opening values  
    idx = np.where(name_array == '{}.csv'.format(j))[0][0]
    predicted_values = pd.DataFrame(obj[1][idx], columns = ['Close', 'Predicted Open', 'High', 'Low'])
    predicted_values = predicted_values.iloc[:,1]
    
    #create plot for the opening values 
    #remove top and rightmost spines to clear things up 
    mpl.rcParams["axes.spines.right"] = False
    mpl.rcParams["axes.spines.top"] = False
    
    #LINES
    plt.plot(range(100), a_valz, label = 'Actual Opening Value', alpha = 0.75)
    plt.plot(range(100), predicted_values, linestyle = (0, (1, 1)),  alpha = 1, label = 'Predicted Opening Value')
    
    #Buy/Sell
    plt.scatter(buy_decisions, dat.iloc[buy_decisions,0], label = 'Time Purchased', marker = '.', alpha = 1, color = 'green')
    plt.scatter(sell_decisions, dat.iloc[sell_decisions,0], label = 'Time Sold', marker = '*', alpha = 0.5, color = 'darkviolet')
    
    #AXES
    plt.title('Actual vs Predicted Opening Values for ${}/USD in Final 100 Hours of Trading'.format(j.split("USD")[0]))
    plt.rcParams['xtick.labelsize']= 10
    plt.xticks(np.arange(0,100,5), labels = final[::5], rotation = 90)
    plt.xlabel('Hour Steps')
    plt.ylabel('Price ($USD)')
    plt.grid(color = 'dimgray', alpha = 0.6)
    plt.legend(bbox_to_anchor=(1, 0.9))
    plt.plot()
    
    plt.savefig('figures/{} Trading.png'.format(j), bbox_inches = 'tight')
    
    plt.close()
    
    
    
    